## Preambule

In [1]:
import numpy as np
from tqdm import tqdm
from importlib import reload
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
from scipy.optimize import curve_fit
from scipy.signal import savgol_filter
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Run classes

In [2]:
import class_datareading
reload(class_datareading)
from class_datareading import datareading

datareader = datareading()
datareader.read_general()
datareader.read_ssps()
datareader.read_undata()
datareader.read_historicalemis()
datareader.read_ar6()
datareader.relation_budget_nonco2()
datareader.determine_global_nonco2_trajectories()
datareader.determine_global_budgets()
datareader.determine_global_co2_trajectories()
datareader.read_baseline()
datareader.read_ndc() 
datareader.merge_xr()
datareader.add_country_groups()
datareader.save()

# ==================================== #
# Initializing datareading class     #
# ==================================== #
- Reading general data
- Reading GDP and population data from SSPs
- Reading UN population data (for past population)
- Reading historical emissions (primap)
- Read AR6 data
- Get relationship between CO2 budgets and non-co2 reduction in 2050
- Computing global nonco2 trajectories
- Get global CO2 budgets
- Computing global co2 trajectories
- Reading baseline emissions
- Reading NDC data
- Merging xrarray object
- Add country groups
- Save important files
- Some pre-calculations for the AP allocation rule
- Some pre-calculations for the GDR allocation rule


In [56]:
import class_allocation

reload(class_allocation)
from class_allocation import allocation

for cty in tqdm(['NLD']):#np.array(datareader.xr_total.Region)):
    allocator = allocation(cty)
    allocator.gf()  
    allocator.pc()
    allocator.pcc()
    allocator.ecpc()
    allocator.ap()
    allocator.gdr()
    allocator.save()

100%|██████████| 1/1 [00:38<00:00, 38.40s/it]


In [10]:
self = datareader
df_raw = pd.read_excel("X:/user/dekkerm/Data/IPCC/CarbonBudgets_IPCC.xlsx", sheet_name='Sheet1')
df = df_raw.rename(columns={'Budget_17': 0.17, 'Budget_33': 0.33, 'Budget_50': 0.50, 'Budget_67': 0.67, 'Budget_83': 0.83})
dummy = df.melt(id_vars=["Temperature"], var_name="Probability", value_name="Budget")
dummy = dummy.set_index(["Temperature", "Probability"])
dummy['Budget'] = dummy['Budget'] - float(self.xr_primap.sel(Region='EARTH', Time=2020).CO2_hist)/1e3
xr_bud_co2 = xr.Dataset.from_dataframe(dummy)

# # Fit nonco2 reduction in forster to different T levels
# tot_2050 = np.zeros(3)
# for T_i, T in enumerate([1.5, 1.7, 2.0]):
#         tot_2050[T_i] = ch4_2020*(1-[0.48, 0.47, 0.35][T_i]) + n2o_2020*(1-[0.25, 0.15, 0.09][T_i])
# forster_red = tot_2050/np.array(tot_2020)
# f_a, f_b = np.polyfit(np.array([1.5, 1.7, 2.0]), forster_red, 1)

def ms_temp(T):
    peaktemp = self.xr_ar6.sel(Variable='AR6 climate diagnostics|Surface Temperature (GSAT)|MAGICCv7.5.3|50.0th Percentile').Value.max(dim='Time')
    return self.xr_ar6.ModelScenario[np.where((peaktemp < T+0.1) & (peaktemp > T-0.1))[0]]
    
# Determine bunker emissions to subtract from global budget
bunker_subtraction = []
for t_i, t in enumerate(self.Tlist):
    ms = ms_temp(t)
    bunker_subtraction += [3.3/100] # Assuming bunker emissions keep constant (3.3% of global emissions) - https://www.pbl.nl/sites/default/files/downloads/pbl-2020-analysing-international-shipping-and-aviation-emissions-projections_4076.pdf

# Interpolate
n2o_2020 = self.xr_primap.sel(Region='EARTH').sel(Time=2020).N2O_hist*self.settings['params']['gwp_n2o']/1e3
ch4_2020 = self.xr_primap.sel(Region='EARTH').sel(Time=2020).CH4_hist*self.settings['params']['gwp_ch4']/1e3
tot_2020 = n2o_2020+ch4_2020
Blist = np.zeros(shape=(len(self.Tlist), len(self.Plist), len(self.NonCO2list)))+np.nan
for p_i, p in enumerate(self.Plist):
    a, b = np.polyfit(xr_bud_co2.Temperature, xr_bud_co2.sel(Probability = np.round(p, 2)).Budget, 1)
    for t_i, t in enumerate(self.Tlist):
        median_budget = (a*t+b)*(1-bunker_subtraction[t_i])
        #implied_leftover_median = f_a*t+f_b
        for n_i, n in enumerate(self.NonCO2list):
                ar = np.array((((self.xr_traj_nonco2.sel(NonCO2red=n, Time=2040, Temperature=t).mean(dim='Timing')-tot_2020) / tot_2020).round(2)).NonCO2_globe)
                ar = ar[~np.isnan(ar)]
                ar[ar > 0] =0
                ar[ar < -0.8] = -0.8
                nonco2effect = self.xr_nonco2effects.sel(Temperature=1.5, NonCO2red=-ar.round(2)).EffectOnRCB
                Blist[t_i, p_i, n_i] = median_budget+nonco2effect#*(n-implied_leftover_median)
                if p == 0.5 and t == 1.5 and n == 0.5: print(median_budget+nonco2effect)
data2 = xr.DataArray(Blist,
                    coords={'Temperature': self.Tlist,
                            'Risk': (1-self.Plist).astype(float).round(2),
                            'NonCO2red': self.NonCO2list},
                    dims=['Temperature', 'Risk', 'NonCO2red'])
self.xr_co2_budgets = xr.Dataset({'Budget': data2})

<xarray.DataArray 'EffectOnRCB' (NonCO2red: 1)>
array([526.9967535])
Coordinates:
    Temperature  float64 1.5
  * NonCO2red    (NonCO2red) float64 0.53


In [6]:
import class_tempalign
reload(class_tempalign)
from class_tempalign import tempaligning

tempaligner = tempaligning() # FIRST RUN AGGREGATOR FOR THIS!! (2030 alloc)
tempaligner.get_relation_2030emis_temp()
tempaligner.determine_tempoutcomes()
tempaligner.save()

# ==================================== #
# Initializing tempaligning class        #
# ==================================== #
- Determine relation between 2030-emissions and temperature outcome
- Determine temperature metric


100%|██████████| 6/6 [00:53<00:00,  8.99s/it]


- Save


In [4]:
import class_policyscens
reload(class_policyscens)
from class_policyscens import policyscenadding

policyscenner = policyscenadding()
policyscenner.read_engage_data()
policyscenner.filter_and_convert()
policyscenner.add_to_xr()

# ==================================== #
# Initializing policyscenadding class  #
# ==================================== #
- Read ENGAGE scenarios and change region namings
- Filter correct scenarios and convert to xarray object
- Add to overall xrobject
